In [38]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader
def pptx_parser(file_path):
    """
    Parses a PowerPoint file and returns its content as a list of documents.

    Args:
        file_path (str): The path to the PowerPoint file.
        
    Returns:
        list: A list of documents extracted from the PowerPoint file.
    """
    loader = UnstructuredPowerPointLoader(file_path)
    documents = loader.load()
    return documents
    

In [39]:
import os
available_extentions = ['pptx', 'ppt']

def parse_file(file_path, extentions=available_extentions):
    """
    Parses a file based on its extension and returns its content.
    """
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    # get the file's extension
    file_extension = file_path.split('.')[-1].lower()
    if file_extension not in extentions:
        raise ValueError(f"Unsupported file extension: {file_extension}")
    if file_extension in ['pptx', 'ppt']:
        return pptx_parser(file_path)
    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

In [ ]:
file_path = "../data/documents/Introduction to generative AI.pptx"
documents = parse_file(file_path)
for doc in documents:
    print(doc)

FileNotFoundError: File not found: ../data/Introduction to generative AI.pptx

In [1]:
# chunking the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
def chunk_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Splits documents into smaller chunks.

    Args:
        documents (list): List of documents to be chunked.
        chunk_size (int): Size of each chunk.
        chunk_overlap (int): Overlap between chunks.
        
    Returns:
        list: List of text chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_documents(documents)

# Chunk the documents
chunked_documents = chunk_documents(documents)
chunked_documents

NameError: name 'documents' is not defined

In [1]:
# indexing the documents using QDrant
from langchain_qdrant import QdrantVectorStore
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
import os, dotenv
from datetime import datetime
dotenv.load_dotenv()

QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6333")
EMBEDDING_MODEL_NAME = "nv-embed-v1"

def index_documents(documents, collection_name="pptx_collection"+str(int(datetime.now().timestamp())), qdrant_url=QDRANT_URL, embedding_model_name=EMBEDDING_MODEL_NAME):
    """
    Indexes documents using QDrant vector store.

    Args:
        documents (list): List of documents to be indexed.
        collection_name (str): Name of the QDrant collection.
        qdrant_url (str): URL of the QDrant instance.
        embedding_model_name (str): Name of the embedding model to use.

    Returns:
        Qdrant: The indexed QDrant vector store.
    """
    vectorstore = QdrantVectorStore.from_documents(
        documents,
        embedding=NVIDIAEmbeddings(model_name=embedding_model_name, nvidia_api_key=os.getenv("NVIDIA_API_KEY")),
        collection_name=collection_name,
        url=qdrant_url,
        prefer_grpc=True,
    )
    return vectorstore

# get vectorstore
def get_vectorstore(collection_name="pptx_collection", qdrant_url=QDRANT_URL, embedding_model_name=EMBEDDING_MODEL_NAME):
    """
    get vectorstore with specified collection name and QDrant URL.

    Returns:
        Qdrant: The QDrant vector store.
    """
    return QdrantVectorStore.from_existing_collection(
        collection_name=collection_name,
        url=qdrant_url,
        embedding=NVIDIAEmbeddings(model_name=embedding_model_name),
    )

In [43]:
# Example usage:
vectorstore = index_documents(chunked_documents, collection_name="genai")
print(f"Indexed {len(chunked_documents)} documents into QDrant collection '{vectorstore.collection_name}'.")

/home/benhima/pi/ai-services/.venv/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:808: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.10.1. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(**client_options)


Indexed 10 documents into QDrant collection 'genai'.


In [2]:
# get the vectorstore
vectorstore = get_vectorstore(collection_name="genai")
print(f"Retrieved vectorstore with collection name: {vectorstore.collection_name}")

/home/benhima/pi/ai-services/.venv/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:397: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.10.1. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(


Retrieved vectorstore with collection name: genai


In [45]:
# =============================================================================
# Multi-Agent Lab Generation Workflow
#
# Roles:
#  - planner_agent: decide QCM vs coding exercise and difficulty level
#  - retriever_agent: split documents, embed chunks, retrieve relevant ones
#  - qcm_generator_agent: generate MCQ questions/options/correct_answer
#  - code_generator_agent: produce coding exercise description and stub
#  - test_generator_agent: create unit tests: input/output pairs
#  - executor_agent: run the code against tests and capture results
#  - evaluator_agent: check test outcomes; if failure, trigger refinement loop
#
# Workflow:
#  1. load_documents(paths: List[str]) -> raw_texts
#  2. chunk_texts(raw_texts) -> chunks
#  3. embed_and_store(chunks) in vector DB
#  4. planner = planner_agent(user_query, metadata)
#     if planner.task == "qcm":
#         chunks = retriever_agent(planner.topic)
#         qcms = qcm_generator_agent(chunks, planner.difficulty)
#         evaluator_agent.validate_qcm(qcms)
#     elif planner.task == "code":
#         chunks = retriever_agent(planner.topic)
#         stub = code_generator_agent(chunks, planner.difficulty)
#         tests = test_generator_agent(stub, chunks)
#         result = executor_agent.run_tests(stub, tests)
#         evaluator_agent.loop_until_pass(stub, tests, result)
#
# Each agent should be implemented as a separate function or class method.
# Use LangChain + LangGraph for orchestration; vector DB for chunk retrieval;
# code sandbox for execution.
#
# Convention:
#  - Clear function signatures for each agent
#  - Use meaningful names and type hints
#  - Keep each agent focused on its responsibility
#
# Goals:
#  - Modular, testable, and easy-to-debug pipeline
#  - Support iterative refinement via evaluator loops
# =============================================================================

In [5]:
"""
    Workflow steps using LangChain, LangGraph & LangSmith:

    1. ✅ Load and preprocess documents
       - Use LangChain document loaders (PDF, PPTX, Markdown, code).
       - Chunk texts with RecursiveCharacterTextSplitter.
       - Embed chunks using OpenAI embeddings.
       - Store in vector DB (e.g., FAISS, Qdrant).

    2. 🧠 Retrieve relevant chunks
       - If user_query provided: use retriever to fetch top-k chunks.
       - Else: pick default chunks or use metadata filters.

    3. 🛠 Planner Agent node (LangGraph)
       - Decide on task type and difficulty.
       - Set up graph path: QCM flow vs Coding exercise flow.

    4. 📋 QCM Generator node
       - Prompt LLM with retrieved chunks + few-shot examples.
       - Output: list of questions, options, correct answers.

    5. 💻 Code Generator node
       - Prompt LLM to generate exercise description & starter code.
       - Use few-shot samples of coding exercises and formats.

    6. 🧪 Test Generator node
       - Prompt LLM to generate unit test cases: inputs & expected outputs.
       - Ensure sufficient coverage across edge cases.

    7. 🚀 Executor node
       - Use a safe code execution environment (sandbox).
       - Run the code stub against unit test cases, capture results.

    8. 🧭 Evaluator node (LangGraph)
       - Check if tests passed. If not:
           • Loop back to Code/Test Generator nodes to refine.
       - Continue until pass or max retries.

    9. ✅ Final output preparation
       - Assemble lab payload: QCM items or coding prompt + tests + pass results.
       - Return structured JSON or object.

    10. ✍️ LangSmith Integration
       - Import tracing via `LANGCHAIN_TRACING_V2="true"` or code decorator.
       - Log node-level runs to LangSmith for observability & debugging.
       - Optionally use LangSmith Evaluators to auto-score generated labs. :contentReference[oaicite:1]{index=1}

    ✅ Use langchain-core + langgraph to build Agent graph and tool connectors :contentReference[oaicite:2]{index=2}
"""
pass

In [6]:
import os, json
from typing import Dict
from langchain_qdrant import Qdrant
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent

MODEL_NAME = "meta/llama-3.3-70b-instruct"

# --- QCM generator using ReAct agent ---
def qcm_pipeline(user_query: str, vectorstore: Qdrant, difficulty: str, context: str, number_of_questions: int) -> Dict:
    llm = ChatNVIDIA(model=MODEL_NAME, nvidia_api_key=os.getenv("NVIDIA_API_KEY"))


    # Pre-fetch context (5 docs max)
    docs = vectorstore.similarity_search(user_query, k=5)
    context_text = "\n".join([d.page_content for d in docs])

    # Prepare prompt with explicit context
    sys_msg = SystemMessage(content=(
        f"You are an educational assistant. Using the context below, generate exactly {number_of_questions} multiple-choice questions at the '{difficulty}' level. "
        f"Each question must have exactly 3 plausible options and ONLY ONE correct answer.\n\n"
        f"Context:\n{context_text}\n{context}\n\n"
        "Return only a valid JSON in this exact format (use double quotes for all keys and string values):\n"
        '''{
    "quiz": [
        {
        "question": "string",
        "options": ["option1", "option2", "option3"],
        "answer": 1
        }
        // Repeat this format for each question
    ]
    }'''
        f"\nYou MUST return exactly {number_of_questions} questions inside the 'quiz' array. No more, no less. Do NOT explain. Only output the JSON."
    ))


    human_msg = HumanMessage(content=f"Generate a question about: {user_query}")

    # No retrieval tool needed here, just pass the LLM
    agent = create_react_agent(model=llm, tools=[], debug=True)

    result = agent.invoke({"messages": [sys_msg, human_msg]})

    try:
        parsed = json.loads(result['messages'][-1].content.strip().strip("```json").strip("```"))
    except json.JSONDecodeError:
        raise ValueError(f"Output is not valid JSON:\n{result['messages'][-1].content}")

    return parsed



In [17]:
qcm = qcm_pipeline("generative AI", vectorstore, difficulty="medium", number_of_questions=10, context="")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [SystemMessage(content='You are an educational assistant. Using the context below, generate exactly 10 multiple-choice questions at the \'medium\' level. Each question must have exactly 3 plausible options and ONLY ONE correct answer.\n\nContext:\nIntroduction to Generative AI\n\n\n\nKhaoula ALLAK\n\nGDG Mentor\n\n\n\nTable of Contents\n\n01\n\nWhat is Generative AI ?\n\n02\n\nFundamentals of Large Language Models \n\n03\n\nHow to customize the LLM  ? \n\n04\n\nPractice\n\n\n\n01\n\nWhat is Generative AI ?\n\n\n\nEvolution of AI \n\nWhat matters\n\n to us today !\n\n\n\nEvolution of AI Use Cases\n\nPredictive AI\n\nGenerative AI\n\nMultimodal\n\nGenerative AI\n\nText, Image & Code Generation\n\nText & Code Rewriting & Formatting\n\nSummarization\n\nExtractive Q&A\n\nImage & Video Descriptions\n\nRegression & Classification\n\nForecasting\n\nSentiment Analysis\

In [18]:
len(qcm['quiz'])

10

In [19]:
qcm['quiz']

[{'question': 'What type of AI is capable of generating text, images, and code?',
  'options': ['Predictive AI', 'Generative AI', 'Multimodal AI'],
  'answer': 1},
 {'question': 'Which of the following is a characteristic of Generative AI?',
  'options': ['Limited to predictive analytics',
   'Only generates text',
   'Can generate text, image, and code'],
  'answer': 2},
 {'question': 'What is the primary function of a Large Language Model (LLM)?',
  'options': ['To recognize and interpret human language',
   'To predict and generate the next word',
   'To analyze and understand visual data'],
  'answer': 0},
 {'question': 'What is an example of a Generative AI application?',
  'options': ['Sentiment analysis',
   'Image classification',
   'Text generation and rewriting'],
  'answer': 2},
 {'question': 'Which company published the Transformer paper in 2017?',
  'options': ['Google', 'Facebook', 'Microsoft'],
  'answer': 0},
 {'question': 'What is the name of the chatbot announced by 

In [39]:
# create a react agent that generate a coding exo question, list of input and expected outputs (to use later for verification)
# to check, is the user code correct or not?
# generate code (solution) for the exo.
# use a tool to run the code with the inputs and verify the outputs to `check if the agent's code or inputs outputs are correct`
# stop when the code is correct, and inputs and outputs are valid,
# return json format {exo: "", solution: "", inputs: [], outputs: []}
# using langchain Sandbox a code execution environment and react agent to generate coding exercises

from langchain_sandbox import PyodideSandboxTool

def coding_exo_pipeline(user_query, vectorstore, difficulty, context):
    # Pre-fetch context (5 docs max)
    docs = vectorstore.similarity_search(user_query, k=5)
    context_text = "\n".join([d.page_content for d in docs])

    tool = PyodideSandboxTool(allow_net=True)


    llm = ChatNVIDIA(model=MODEL_NAME, nvidia_api_key=os.getenv("NVIDIA_API_KEY"))
    sys_msg = SystemMessage(content=(
        f"You are an expert Python coding exercise generator and validator.\n"
        f"Given the context below, do the following:\n"
        f"1. Generate a Python coding exercise at the '{difficulty}' level.\n"
        f"2. Provide a correct Python solution for the exercise.\n"
        f"3. Create at least 3 test cases (inputs and expected outputs).\n"
        f"4. Use the solution you generated to execute each input using the PyodideSandboxTool.\n"
        f"5. Compare the actual outputs from execution with your expected outputs.\n"
        f"6. If they don't match, regenerate the solution and/or the outputs until they match exactly.\n"
        f"7. Ensure a strict 1:1 mapping between inputs and outputs (same length, same order).\n"
        f"8. Return only a valid JSON. Do not include explanations, markdown, or additional text.\n\n"
        f"Context:\n{context_text}\n{context}\n\n"
        "Output format:\n"
        '''{
    "exercise": "string",
    "solution": "string",
    "inputs": [[...], [...], ...],
    "outputs": [...]
    }'''
        "\nInputs and outputs must be matched correctly. Each input corresponds to exactly one output at the same index. Only output the JSON response."
    ))



    human_msg = HumanMessage(content=f"Generate a coding exercise about: {user_query}")

    # No retrieval tool needed here, just pass the LLM
    agent = create_react_agent(model=llm, tools=[tool], debug=True)

    result = agent.invoke({"messages": [sys_msg, human_msg]})

    try:
        parsed = json.loads(result['messages'][-1].content.strip().strip("```json").strip("```"))
    except json.JSONDecodeError:
        raise ValueError(f"Output is not valid JSON:\n{result['messages'][-1].content}")

    return parsed

In [13]:
# instlall deno in your terminal: curl -fsSL https://deno.land/install.sh | sh
# export DENO_INSTALL="$HOME/.deno"
# export PATH="$DENO_INSTALL/bin:$PATH"
# source ~/.bashrc 
# deno --version
# then run this command if it fails. quit you ide or reopen it and execute this cell.
import subprocess
print(subprocess.run(["deno", "--version"], capture_output=True).stdout.decode())

deno 2.3.5 (stable, release, x86_64-unknown-linux-gnu)
v8 13.7.152.6-rusty
typescript 5.8.3



In [40]:
result = coding_exo_pipeline("build a matrix multiplication", vectorstore, "easy", "")
result

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [SystemMessage(content='You are an expert Python coding exercise generator and validator.\nGiven the context below, do the following:\n1. Generate a Python coding exercise at the \'easy\' level.\n2. Provide a correct Python solution for the exercise.\n3. Create at least 3 test cases (inputs and expected outputs).\n4. Use the solution you generated to execute each input using the PyodideSandboxTool.\n5. Compare the actual outputs from execution with your expected outputs.\n6. If they don\'t match, regenerate the solution and/or the outputs until they match exactly.\n7. Ensure a strict 1:1 mapping between inputs and outputs (same length, same order).\n8. Return only a valid JSON. Do not include explanations, markdown, or additional text.\n\nContext:\nSolve this problem step by step: If a car travels 60 miles in 2 hours, what is its speed? Here’s how you solve it

{'exercise': 'Build a function to perform matrix multiplication on two 2x2 matrices.',
 'solution': 'def matrix_multiplication(A, B):\n    if len(A[0]) != len(B):\n        return "Incompatible matrices for multiplication"\n    result = [[0, 0],\n              [0, 0]]\n    for i in range(len(A)):\n        for j in range(len(B[0])):\n            for k in range(len(B)):\n                result[i][j] += A[i][k] * B[k][j]\n    return result',
 'inputs': [[[1, 2], [3, 4]], [[5, 6], [7, 8]]],
 'outputs': [[19, 22], [43, 50]]}

In [41]:
print(result['solution'])

def matrix_multiplication(A, B):
    if len(A[0]) != len(B):
        return "Incompatible matrices for multiplication"
    result = [[0, 0],
              [0, 0]]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result


In [46]:
def matrix_multiplication(A, B):
    if len(A[0]) != len(B):
        return "Incompatible matrices for multiplication"
    result = [[0, 0],
              [0, 0]]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

# Test the solution with the provided inputs
inputs = result['inputs']
outputs = result['outputs']

# check the inputs and outputs
outputs_res = matrix_multiplication(*inputs)
outputs_res

[[19, 22], [43, 50]]

In [ ]:
from typing import List
def generate_lab(context: str, number_of_question: int, user_query: str=None, task: str="qcm", difficulty: str="easy", files_paths: List[str]=[]):
    if(task not in ["qcm", "code"]):
        raise ValueError("Invalid task type. Choose 'qcm' or 'code'.")
    # Load and parse documents
    documents = []
    for file_path in files_paths:
        try:
            docs = parse_file(file_path)
            documents.extend(docs)
        except Exception as e:
            print(f"Error parsing {file_path}: {e}")
    if not documents:
        raise ValueError("No valid documents found to process.")
    # Chunk the documents
    chunked_documents = chunk_documents(documents)
    if not chunked_documents:
        raise ValueError("No valid chunks created from documents.")
    collection_name = files_paths[0].split('/')[-1].split('.')[0] + "_collection"
    # Index the documents
    vectorstore = index_documents(chunked_documents, collection_name=collection_name)
    # use the appropriate agent based on the task
    if task == "qcm":
        # results is in a dict
        results = qcm_pipeline(user_query, vectorstore, difficulty, context, number_of_question)
    else:
        results = coding_exo_pipeline(user_query, vectorstore, difficulty, context)
    return results

In [ ]:
{
    'context': Optional[str],
    'user_query': str,
    'type': str,  # 'qcm' or 'code'
    'difficulty': str,  # 'easy', 'medium', 'hard'
    'number_of_questions': int,  # number of questions to generate
    'files_paths': Optional[List[str]],  # paths to files to parse
}

# ila kan task == "qcm":
{
"quiz": [
    {
    "question": "string",
    "options": ["option1", "option2", "option3"],
    "answer": 1
    }
]
}

# ila kan task == "code":
{
    'question': str,  # description of the coding exercise
    'solution': str,  # code solution
    'inputs': List[str],  # list of inputs for the code
    'outputs': List[str]  # expected outputs for the inputs
}

{}